# SAC Ion Exchange System Analysis Report

Generated: `{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}`

This report presents the analysis results for a Strong Acid Cation (SAC) ion exchange system based on the provided configuration parameters.

In [ ]:
# Parameters cell - will be replaced by papermill
water_analysis = {
    "flow_m3_hr": 100.0,
    "temperature_celsius": 25.0,
    "pH": 7.5,
    "pressure_bar": 1.0,
    "ca_mg_l": 80.0,
    "mg_mg_l": 25.0,
    "na_mg_l": 800.0,
    "cl_mg_l": 1400.0,
    "hco3_mg_l": 120.0
}

vessel_configuration = {
    "bed_volume_L": 1000.0,
    "service_flow_rate_bv_hr": 16.0,
    "vessel_id": "SAC-001",
    "number_service": 1,
    "number_standby": 1,
    "diameter_m": 1.5,
    "bed_depth_m": 2.0,
    "resin_volume_m3": 1.0,
    "freeboard_m": 0.5,
    "vessel_height_m": 2.5
}

target_hardness_mg_l_caco3 = 5.0

regeneration_config = {
    "enabled": True,
    "regenerant_type": "NaCl",
    "concentration_percent": 11.0,
    "flow_rate_bv_hr": 2.5,
    "mode": "staged_optimize",
    "regeneration_stages": 5,
    "target_recovery": 0.90
}

# Project root parameter (passed by papermill for path resolution)
project_root_str = None

In [ ]:
# Imports and setup
import sys
import os
from pathlib import Path
import json
import pandas as pd
import numpy as np
from datetime import datetime
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Add project root to path - robust approach
if 'IX_DESIGN_MCP_ROOT' in os.environ:
    project_root = Path(os.environ['IX_DESIGN_MCP_ROOT'])
elif 'project_root_str' in locals():
    # Parameter passed by papermill
    project_root = Path(project_root_str)
else:
    # Fallback: assume notebook is in notebooks/ directory
    try:
        project_root = Path(__file__).parent.parent
    except NameError:
        # __file__ not defined in Jupyter, use cwd fallback
        project_root = Path.cwd().parent if 'notebooks' in str(Path.cwd()) else Path.cwd()

if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Import simulation and plotting functions
from tools.sac_simulation import simulate_sac_phreeqc, SACSimulationInput, RegenerationConfig
from tools.sac_configuration import SACWaterComposition, SACVesselConfiguration

# Professional color scheme
colors = {
    'primary': '#1f77b4',     # Blue
    'secondary': '#ff7f0e',   # Orange  
    'tertiary': '#2ca02c',    # Green
    'quaternary': '#d62728',  # Red
    'grid': '#E5E5E5',
    'background': '#FAFAFA'
}

print("Report generation initialized")
print(f"Project root: {project_root}")

## 1. Configuration Summary

### 1.1 Water Analysis

In [ ]:
# Create water analysis summary table
water_df = pd.DataFrame([
    {'Parameter': 'Flow Rate', 'Value': water_analysis['flow_m3_hr'], 'Unit': 'm³/hr'},
    {'Parameter': 'Temperature', 'Value': water_analysis['temperature_celsius'], 'Unit': '°C'},
    {'Parameter': 'pH', 'Value': water_analysis['pH'], 'Unit': '-'},
    {'Parameter': 'Pressure', 'Value': water_analysis['pressure_bar'], 'Unit': 'bar'},
    {'Parameter': 'Calcium (Ca²⁺)', 'Value': water_analysis['ca_mg_l'], 'Unit': 'mg/L'},
    {'Parameter': 'Magnesium (Mg²⁺)', 'Value': water_analysis['mg_mg_l'], 'Unit': 'mg/L'},
    {'Parameter': 'Sodium (Na⁺)', 'Value': water_analysis['na_mg_l'], 'Unit': 'mg/L'},
    {'Parameter': 'Chloride (Cl⁻)', 'Value': water_analysis['cl_mg_l'], 'Unit': 'mg/L'},
    {'Parameter': 'Bicarbonate (HCO₃⁻)', 'Value': water_analysis['hco3_mg_l'], 'Unit': 'mg/L'},
])

# Calculate total hardness
total_hardness = water_analysis['ca_mg_l'] / 0.4008 + water_analysis['mg_mg_l'] / 0.2428
water_df = pd.concat([water_df, pd.DataFrame([{
    'Parameter': 'Total Hardness (as CaCO₃)',
    'Value': f"{total_hardness:.1f}",
    'Unit': 'mg/L'
}])], ignore_index=True)

display(water_df.style.set_properties(**{'text-align': 'left'}).hide(axis='index'))

### 1.2 Vessel Configuration

In [ ]:
# Create vessel configuration summary table
vessel_df = pd.DataFrame([
    {'Parameter': 'Vessel ID', 'Value': vessel_configuration['vessel_id'], 'Unit': '-'},
    {'Parameter': 'Bed Volume', 'Value': vessel_configuration['bed_volume_L'], 'Unit': 'L'},
    {'Parameter': 'Bed Depth', 'Value': vessel_configuration['bed_depth_m'], 'Unit': 'm'},
    {'Parameter': 'Vessel Diameter', 'Value': vessel_configuration['diameter_m'], 'Unit': 'm'},
    {'Parameter': 'Service Flow Rate', 'Value': vessel_configuration['service_flow_rate_bv_hr'], 'Unit': 'BV/hr'},
    {'Parameter': 'Number in Service', 'Value': vessel_configuration['number_service'], 'Unit': '-'},
    {'Parameter': 'Number in Standby', 'Value': vessel_configuration['number_standby'], 'Unit': '-'},
    {'Parameter': 'Target Effluent Hardness', 'Value': target_hardness_mg_l_caco3, 'Unit': 'mg/L CaCO₃'},
])

display(vessel_df.style.set_properties(**{'text-align': 'left'}).hide(axis='index'))

### 1.3 Regeneration Configuration

In [ ]:
# Create regeneration configuration summary table
regen_df = pd.DataFrame([
    {'Parameter': 'Regeneration Enabled', 'Value': regeneration_config['enabled'], 'Unit': '-'},
    {'Parameter': 'Regenerant Type', 'Value': regeneration_config['regenerant_type'], 'Unit': '-'},
    {'Parameter': 'Concentration', 'Value': regeneration_config['concentration_percent'], 'Unit': '%'},
    {'Parameter': 'Flow Rate', 'Value': regeneration_config['flow_rate_bv_hr'], 'Unit': 'BV/hr'},
    {'Parameter': 'Mode', 'Value': regeneration_config['mode'], 'Unit': '-'},
    {'Parameter': 'Number of Stages', 'Value': regeneration_config['regeneration_stages'], 'Unit': '-'},
])

if 'target_recovery' in regeneration_config:
    regen_df = pd.concat([regen_df, pd.DataFrame([{
        'Parameter': 'Target Recovery',
        'Value': f"{regeneration_config['target_recovery']*100:.0f}",
        'Unit': '%'
    }])], ignore_index=True)

display(regen_df.style.set_properties(**{'text-align': 'left'}).hide(axis='index'))

## 2. Simulation Results

In [ ]:
# Create input objects and run simulation
water = SACWaterComposition(**water_analysis)
vessel = SACVesselConfiguration(**vessel_configuration)
regen = RegenerationConfig(**regeneration_config)

sim_input = SACSimulationInput(
    water_analysis=water,
    vessel_configuration=vessel,
    target_hardness_mg_l_caco3=target_hardness_mg_l_caco3,
    regeneration_config=regen
)

# Run simulation
print("Running PHREEQC simulation...")
results = simulate_sac_phreeqc(sim_input)
print("Simulation completed successfully")

### 2.1 Service Phase Results

In [ ]:
# Service phase results table
service_results = pd.DataFrame([
    {'Metric': 'Breakthrough Volume', 'Value': f"{results.breakthrough_bv:.1f}", 'Unit': 'BV'},
    {'Metric': 'Service Time', 'Value': f"{results.service_time_hours:.1f}", 'Unit': 'hours'},
    {'Metric': 'Treated Volume', 'Value': f"{results.breakthrough_bv * vessel.bed_volume_L / 1000:.1f}", 'Unit': 'm³'},
    {'Metric': 'Capacity Utilization Factor', 'Value': f"{results.phreeqc_determined_capacity_factor:.3f}", 'Unit': '-'},
    {'Metric': 'Capacity Utilization', 'Value': f"{results.capacity_utilization_percent:.1f}", 'Unit': '%'},
])

display(service_results.style.set_properties(**{'text-align': 'left'}).hide(axis='index'))

### 2.2 Regeneration Results

In [ ]:
# Regeneration results table
if results.regeneration_results:
    regen_results = pd.DataFrame([
        {'Metric': 'Regenerant Consumed', 'Value': f"{results.regeneration_results.regenerant_consumed_kg:.1f}", 'Unit': 'kg'},
        {'Metric': 'Final Resin Recovery', 'Value': f"{results.regeneration_results.final_resin_recovery * 100:.1f}", 'Unit': '%'},
        {'Metric': 'Regeneration Time', 'Value': f"{results.regeneration_results.regeneration_time_hours:.1f}", 'Unit': 'hours'},
        {'Metric': 'Regenerant Dose', 'Value': f"{results.regeneration_results.regenerant_consumed_kg / (vessel.bed_volume_L/1000) :.1f}", 'Unit': 'kg/m³ resin'},
    ])
    display(regen_results.style.set_properties(**{'text-align': 'left'}).hide(axis='index'))
else:
    print("Regeneration not performed")

### 2.3 Complete Cycle Summary

In [ ]:
# Complete cycle summary
cycle_summary = pd.DataFrame([
    {'Metric': 'Total Cycle Time', 'Value': f"{results.total_cycle_time_hours:.1f}", 'Unit': 'hours'},
    {'Metric': 'Service Time Fraction', 'Value': f"{(results.service_time_hours / results.total_cycle_time_hours * 100):.1f}", 'Unit': '%'},
    {'Metric': 'Cycles per Year', 'Value': f"{8760 / results.total_cycle_time_hours:.1f}", 'Unit': 'cycles/year'},
    {'Metric': 'Annual Treated Volume', 'Value': f"{(8760 / results.total_cycle_time_hours) * results.breakthrough_bv * vessel.bed_volume_L / 1000:.0f}", 'Unit': 'm³/year'},
])

if results.regeneration_results:
    annual_regenerant = (8760 / results.total_cycle_time_hours) * results.regeneration_results.regenerant_consumed_kg
    cycle_summary = pd.concat([cycle_summary, pd.DataFrame([{
        'Metric': 'Annual Regenerant Usage',
        'Value': f"{annual_regenerant:.0f}",
        'Unit': 'kg/year'
    }])], ignore_index=True)

display(cycle_summary.style.set_properties(**{'text-align': 'left'}).hide(axis='index'))

## 3. Service Phase Performance Analysis

### Figure 1: Service Phase Performance Dashboard

In [ ]:
# Extract service phase data
bd = results.breakthrough_data
service_mask = [p == 'SERVICE' for p in bd['phases']]
service_indices = [i for i, m in enumerate(service_mask) if m]

if service_indices:
    service_bv = [bd['bed_volumes'][i] for i in service_indices]
    service_ca = [bd['ca_mg_l'][i] for i in service_indices]
    service_mg = [bd['mg_mg_l'][i] for i in service_indices]
    service_na = [bd['na_mg_l'][i] for i in service_indices]
    service_hardness = [bd['hardness_mg_l'][i] for i in service_indices]
    
    # Create subplots
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=(
            'Ion Breakthrough Curves',
            'Total Hardness vs Target',
            'Chromatographic Separation (Mg Region)',
            'Initial Na Release (Regenerant Flush)'
        ),
        specs=[
            [{}, {}],
            [{}, {}]
        ]
    )
    
    # 1. Ion breakthrough curves
    fig.add_trace(
        go.Scatter(x=service_bv, y=service_ca, name='Ca²⁺', line=dict(color=colors['primary'], width=2)),
        row=1, col=1
    )
    fig.add_trace(
        go.Scatter(x=service_bv, y=service_mg, name='Mg²⁺', line=dict(color=colors['tertiary'], width=2)),
        row=1, col=1
    )
    fig.add_trace(
        go.Scatter(x=service_bv, y=service_na, name='Na⁺', line=dict(color=colors['secondary'], width=2)),
        row=1, col=1
    )
    
    # Add breakthrough line
    breakthrough_bv = bd.get('breakthrough_bv', results.breakthrough_bv)
    fig.add_vline(x=breakthrough_bv, line_dash="dash", line_color="red", 
                  annotation_text=f"Breakthrough<br>{breakthrough_bv:.1f} BV",
                  annotation_position="top right", row=1, col=1)
    
    # 2. Hardness vs target
    fig.add_trace(
        go.Scatter(x=service_bv, y=service_hardness, name='Total Hardness',
                   line=dict(color='black', width=2)),
        row=1, col=2
    )
    fig.add_hline(y=target_hardness_mg_l_caco3, line_dash="dot", line_color="red",
                  annotation_text=f"Target: {target_hardness_mg_l_caco3} mg/L",
                  annotation_position="right", row=1, col=2)
    fig.add_vline(x=breakthrough_bv, line_dash="dash", line_color="red", row=1, col=2)
    
    # 3. Mg spike region (chromatographic separation)
    mg_region_start = max(0, breakthrough_bv * 0.9)
    mg_region_end = min(max(service_bv), breakthrough_bv * 1.6)
    mg_region_indices = [i for i, bv in enumerate(service_bv) if mg_region_start <= bv <= mg_region_end]
    
    if mg_region_indices:
        mg_region_bv = [service_bv[i] for i in mg_region_indices]
        mg_region_mg = [service_mg[i] for i in mg_region_indices]
        mg_region_ca = [service_ca[i] for i in mg_region_indices]
        
        fig.add_trace(
            go.Scatter(x=mg_region_bv, y=mg_region_mg, name='Mg²⁺ (zoom)',
                       line=dict(color=colors['tertiary'], width=2)),
            row=2, col=1
        )
        fig.add_trace(
            go.Scatter(x=mg_region_bv, y=mg_region_ca, name='Ca²⁺ (zoom)',
                       line=dict(color=colors['primary'], width=2)),
            row=2, col=1
        )
        fig.add_hline(y=water.mg_mg_l, line_dash="dot", line_color=colors['tertiary'],
                      annotation_text=f"Feed Mg: {water.mg_mg_l} mg/L",
                      annotation_position="right", row=2, col=1)
    
    # 4. Initial Na spike
    na_spike_indices = [i for i, bv in enumerate(service_bv) if bv <= 10]
    if na_spike_indices:
        na_spike_bv = [service_bv[i] for i in na_spike_indices]
        na_spike_na = [service_na[i] for i in na_spike_indices]
        
        fig.add_trace(
            go.Scatter(x=na_spike_bv, y=na_spike_na, name='Na⁺ (initial)',
                       line=dict(color=colors['secondary'], width=2)),
            row=2, col=2
        )
        fig.add_hline(y=water.na_mg_l, line_dash="dot", line_color=colors['secondary'],
                      annotation_text=f"Feed Na: {water.na_mg_l} mg/L",
                      annotation_position="right", row=2, col=2)
    
    # Update layout
    fig.update_xaxes(title_text="Bed Volumes", gridcolor=colors['grid'])
    fig.update_yaxes(title_text="Concentration (mg/L)", gridcolor=colors['grid'])
    fig.update_layout(
        height=800,
        showlegend=True,
        plot_bgcolor=colors['background'],
        title_text="Service Phase Performance Analysis",
        title_x=0.5
    )
    
    fig.show()
else:
    print("No service phase data available")

### Figure 2: Ion Composition Evolution

In [ ]:
# Create stacked area chart for ion composition
if service_indices:
    fig_comp = go.Figure()
    
    # Add stacked areas
    fig_comp.add_trace(go.Scatter(
        x=service_bv, y=service_ca,
        mode='lines',
        name='Ca²⁺',
        line=dict(width=0),
        fillcolor=colors['primary'],
        stackgroup='one'
    ))
    
    fig_comp.add_trace(go.Scatter(
        x=service_bv, y=service_mg,
        mode='lines',
        name='Mg²⁺',
        line=dict(width=0),
        fillcolor=colors['tertiary'],
        stackgroup='one'
    ))
    
    # Add breakthrough line
    fig_comp.add_vline(x=breakthrough_bv, line_dash="dash", line_color="red",
                       annotation_text=f"Target Hardness<br>Breakthrough",
                       annotation_position="top")
    
    fig_comp.update_layout(
        title="Figure 2: Effluent Ion Composition During Service",
        xaxis_title="Bed Volumes",
        yaxis_title="Concentration (mg/L)",
        height=400,
        plot_bgcolor=colors['background'],
        xaxis=dict(gridcolor=colors['grid']),
        yaxis=dict(gridcolor=colors['grid'])
    )
    
    fig_comp.show()

## 4. Regeneration Performance Analysis

In [ ]:
# Regeneration analysis if available
if results.regeneration_results and hasattr(results.regeneration_results, 'stage_results'):
    stages = results.regeneration_results.stage_results
    
    if stages:
        # Extract stage data
        stage_nums = list(range(1, len(stages) + 1))
        recoveries = [s.get('recovery', 0) * 100 for s in stages]
        na_fractions = [s.get('na_fraction', 0) for s in stages]
        
        # Create regeneration dashboard
        fig_regen = make_subplots(
            rows=2, cols=2,
            subplot_titles=(
                'Recovery Progress by Stage',
                'Resin Na Fraction by Stage',
                'Stage Recovery Increment',
                'Regeneration Efficiency'
            ),
            specs=[[{}, {}], [{"type": "bar"}, {}]]
        )
        
        # 1. Recovery progress
        fig_regen.add_trace(
            go.Scatter(x=stage_nums, y=recoveries, mode='lines+markers',
                       name='Recovery', line=dict(color=colors['primary'], width=2)),
            row=1, col=1
        )
        fig_regen.add_hline(y=90, line_dash="dot", line_color="green",
                            annotation_text="90% Target", row=1, col=1)
        
        # 2. Na fraction
        fig_regen.add_trace(
            go.Scatter(x=stage_nums, y=na_fractions, mode='lines+markers',
                       name='Na Fraction', line=dict(color=colors['secondary'], width=2)),
            row=1, col=2
        )
        
        # 3. Stage increments
        increments = [recoveries[0]] + [recoveries[i] - recoveries[i-1] for i in range(1, len(recoveries))]
        fig_regen.add_trace(
            go.Bar(x=stage_nums, y=increments, name='Recovery Increment',
                   marker_color=colors['tertiary']),
            row=2, col=1
        )
        
        # 4. Efficiency curve (if optimization data available)
        # For now, show regenerant usage
        regen_dose = results.regeneration_results.regenerant_consumed_kg / (vessel.bed_volume_L/1000)
        fig_regen.add_trace(
            go.Scatter(x=[regen_dose], y=[recoveries[-1]], mode='markers',
                       marker=dict(size=15, color=colors['quaternary']),
                       name='Operating Point'),
            row=2, col=2
        )
        
        # Update layout
        fig_regen.update_xaxes(title_text="Stage Number", row=1, col=1)
        fig_regen.update_xaxes(title_text="Stage Number", row=1, col=2)
        fig_regen.update_xaxes(title_text="Stage Number", row=2, col=1)
        fig_regen.update_xaxes(title_text="Regenerant Dose (kg/m³)", row=2, col=2)
        
        fig_regen.update_yaxes(title_text="Recovery (%)", row=1, col=1)
        fig_regen.update_yaxes(title_text="Na Fraction", row=1, col=2)
        fig_regen.update_yaxes(title_text="Recovery Increment (%)", row=2, col=1)
        fig_regen.update_yaxes(title_text="Recovery (%)", row=2, col=2)
        
        fig_regen.update_layout(
            height=800,
            title_text="Figure 3: Regeneration Performance Analysis",
            title_x=0.5,
            showlegend=False,
            plot_bgcolor=colors['background']
        )
        fig_regen.update_xaxes(gridcolor=colors['grid'])
        fig_regen.update_yaxes(gridcolor=colors['grid'])
        
        fig_regen.show()
else:
    print("Detailed regeneration data not available")

## 5. Cycle Time Distribution

In [ ]:
# Create cycle time distribution chart
cycle_times = {
    'Service': results.service_time_hours,
    'Regeneration': results.total_cycle_time_hours - results.service_time_hours
}

fig_cycle = go.Figure(data=[go.Pie(
    labels=list(cycle_times.keys()),
    values=list(cycle_times.values()),
    hole=.3,
    marker_colors=[colors['primary'], colors['secondary']]
)])

fig_cycle.update_traces(
    textposition='inside',
    textinfo='percent+label',
    hovertemplate='%{label}: %{value:.1f} hours<br>%{percent}<extra></extra>'
)

fig_cycle.update_layout(
    title="Figure 4: Cycle Time Distribution",
    height=400,
    showlegend=True,
    annotations=[dict(text=f'Total<br>{results.total_cycle_time_hours:.1f} hrs',
                      x=0.5, y=0.5, font_size=16, showarrow=False)]
)

fig_cycle.show()

## 6. Annual Projections

In [ ]:
# Calculate annual projections
cycles_per_year = 8760 / results.total_cycle_time_hours
annual_treated = cycles_per_year * results.breakthrough_bv * vessel.bed_volume_L / 1000

projections = [
    {'Parameter': 'Operating Cycles', 'Annual Value': f"{cycles_per_year:.1f}", 'Unit': 'cycles/year'},
    {'Parameter': 'Treated Water Volume', 'Annual Value': f"{annual_treated:,.0f}", 'Unit': 'm³/year'},
    {'Parameter': 'Service Hours', 'Annual Value': f"{cycles_per_year * results.service_time_hours:,.0f}", 'Unit': 'hours/year'},
]

if results.regeneration_results:
    annual_regen = cycles_per_year * results.regeneration_results.regenerant_consumed_kg
    regen_hours = cycles_per_year * results.regeneration_results.regeneration_time_hours
    
    projections.extend([
        {'Parameter': 'Regenerant Consumption', 'Annual Value': f"{annual_regen:,.0f}", 'Unit': 'kg/year'},
        {'Parameter': 'Regeneration Hours', 'Annual Value': f"{regen_hours:,.0f}", 'Unit': 'hours/year'},
    ])

projections_df = pd.DataFrame(projections)
display(projections_df.style.set_properties(**{'text-align': 'left'}).hide(axis='index'))

## 7. Detailed Breakthrough Data

In [ ]:
# Create detailed breakthrough data table (first 20 and last 20 points)
if service_indices:
    # Sample data points
    sample_indices = service_indices[:10] + service_indices[-10:] if len(service_indices) > 20 else service_indices
    
    breakthrough_sample = pd.DataFrame({
        'Bed Volumes': [f"{bd['bed_volumes'][i]:.1f}" for i in sample_indices],
        'Ca (mg/L)': [f"{bd['ca_mg_l'][i]:.2f}" for i in sample_indices],
        'Mg (mg/L)': [f"{bd['mg_mg_l'][i]:.2f}" for i in sample_indices],
        'Na (mg/L)': [f"{bd['na_mg_l'][i]:.1f}" for i in sample_indices],
        'Hardness (mg/L CaCO₃)': [f"{bd['hardness_mg_l'][i]:.2f}" for i in sample_indices]
    })
    
    print("Table 1: Sample Breakthrough Data (First 10 and Last 10 Service Points)")
    display(breakthrough_sample.style.set_properties(**{'text-align': 'center'}).hide(axis='index'))

## 8. Report Summary

This analysis report presents the simulation results for the configured SAC ion exchange system. All data shown is derived directly from the PHREEQC simulation outputs based on the input parameters provided.

In [ ]:
# Store results for extraction
notebook_results = {
    'status': 'success',
    'breakthrough_bv': results.breakthrough_bv,
    'service_time_hours': results.service_time_hours,
    'total_cycle_time_hours': results.total_cycle_time_hours,
    'capacity_factor': results.phreeqc_determined_capacity_factor,
    'annual_treated_m3': annual_treated,
    'cycles_per_year': cycles_per_year
}

if results.regeneration_results:
    notebook_results.update({
        'final_recovery': results.regeneration_results.final_resin_recovery,
        'regenerant_kg': results.regeneration_results.regenerant_consumed_kg,
        'annual_regenerant_kg': cycles_per_year * results.regeneration_results.regenerant_consumed_kg
    })

# Store as JSON for extraction
notebook_results_json = json.dumps(notebook_results)
print(f"\nAnalysis completed at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")